In [ ]:
#webscraper tool
from selenium import webdriver
#Technically unneccesary, but I like to time how long some operations take
from datetime import datetime
# For cleaning up data
import pandas as pd
# for handling .txt file
import os

In [ ]:
# Instantiates the driver
driver = webdriver.Chrome()

In [ ]:
#I set the root of all the databases as the base for selenium so I #can recycle the code to scrape the other databases
home = 'http://www2.scc.rutgers.edu/memdb/database_list.html'
#Tells selenium to go to the base url
driver.get(home)

In [ ]:
#This is the name of the link text in the first link
first_link = 'Prices (Metz)'

#This is the command that tells Selenium to click the first link:
driver.find_element_by_link_text(first_link).click()

In [ ]:
#found by using the copy xpath tool in the browser inspector
second_link = '/html/body/main/table/tbody/tr/td[2]/form/p/input'

#clicks the button
driver.find_element_by_xpath(second_link).click()

In [ ]:
# name of our text file
filename = 'Metz_prices_demo.txt'
# creates .txt file
file_object  = open(filename, "w+") 

In [ ]:
# Starts our counter for our while loop
counter = 0
# Sets our file to have table data appended to it
export = open(filename, 'a+')
# Optional, but I like to time how long certain functions take
start = datetime.now()

# While loop that goes through each page and saves the data
while counter < 402:
    # The location of the table on every page
    table_loc = '/html/body/form/main/table/tbody/tr/td[2]/p[3]/table/tbody'
    # The selenium function that gets the data from the table and loads it into a variable
    page_text = driver.find_element_by_xpath(table_loc).text
    # The function that writes the table data into our .txt file
    export.write(page_text + '\n')
    try:
        # More common page layout
        button_path = '/html/body/form/main/table/tbody/tr/td[2]/center[3]/input[2]'
        driver.find_element_by_xpath(button_path).click()
    except:
        # Less common page layout
        button_path = '/html/body/form/main/table/tbody/tr/td[2]/center[3]/input'
        driver.find_element_by_xpath(button_path).click()
    # Adds 1 to the counter
    counter += 1

# Timestamps the end of the loop
end = datetime.now()
# Calculates the duration of the loop
duration = end - start

#Prints the duration
print(duration)

In [ ]:
column_names = ['Num', 'Year', 'Month', 'Week', 'Product', 'Number_of_Malters_Sold', 'Currency', 'Price_per_Malter']
# Opens the .txt, uses the two space delimeter, skips the first 3 rows, and sets the column names to the list of names above
df = pd.read_csv('Metz_prices_demo.txt', sep = '  ', skiprows = 3, names = column_names, header = None)
df

In [ ]:
#Shows a header row that we need to clean
df.iloc[[100]]

In [ ]:
# Removes all rows from the dataframe where the contents of the 'Num' column have letters in them
df = df[~df['Num'].str.contains("[a-zA-Z]").fillna(False)]
df

In [ ]:
# Removes duplicate rows that we might have accidentally grabbed from the data
df.drop_duplicates(subset = ['Num'], keep = 'first', inplace = True)
df.reset_index(inplace = True)

In [ ]:
df

In [ ]:
# exports the clean dataframe to a csv
df.to_csv('koln_grain_prices.csv')